In [1]:
## import libraries
from stitch import *
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os, threading, time, multiprocessing, sys, json, math, zipfile, fnmatch, shutil, pickle, logging

CSPICE_N0067


In [2]:
## set directory - desired perijove to be processed (make sure STITCHING/KERNELS/CURRENT contains proper kernels)
perijove = 20
os.chdir("../DATA/RAW/PERIJOVE-" + str(perijove))
print("Current perijove directory: " + os.getcwd())

Current perijove directory: /home/uzumochi/eigenjuno/DATA/RAW/PERIJOVE-20


In [7]:
## unzip junocam website data - just drop zip files in perijove directory and run this block!
for file in sorted(os.listdir('.')):
    meta_num = file.split('-')[0]
    if fnmatch.fnmatch(file, '*-Data.zip') and not os.path.isfile(meta_num + '-Metadata.json'):
        print('Unzipped', meta_num, 'image and data')
        with zipfile.ZipFile(file, 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                zip_ref.extract(n)
        with zipfile.ZipFile(meta_num + '-ImageSet.zip', 'r') as zip_ref:
            names = zip_ref.namelist()
            for n in names:
                if 'raw' in n:
                    zip_ref.extract(n)
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*.zip') or fnmatch.fnmatch(file, '*Zone.Identifier'):
        os.remove(file)

if os.path.isdir('DataSet'):
    for file in sorted(os.listdir('DataSet')):
        shutil.move('DataSet/' + file, file)
    os.rmdir('DataSet')

if os.path.isdir('ImageSet'):
    for file in sorted(os.listdir('ImageSet')):
        shutil.move('ImageSet/' + file, file)
    os.rmdir('ImageSet')

Unzipped 6881 image and data
Unzipped 6882 image and data
Unzipped 6884 image and data
Unzipped 6922 image and data
Unzipped 6923 image and data


In [8]:
## stitch function for thread, taken largely from the reddit post linked in repo desc
def stitch(meta_num):
    x_dim = 2048
    y_dim = 2048
    im_info = meta_num + '-Metadata.json'
    with open(im_info, 'r') as json_file:
        im_info_dir = json.load(json_file)
        image = im_info_dir['FILE_NAME']
        image_time = im_info_dir['IMAGE_TIME']
    
    img = Image.open(image)
    im_ar = np.array(img)
    
    s1, s2 = im_ar.shape

    start_time = im_info_dir["START_TIME"]
    frame_delay = float(im_info_dir["INTERFRAME_DELAY"].split()[0]) + 0.001
    
    start_correction, frame_delay = correct_image_start_time_and_frame_delay(im_ar, start_time, frame_delay)
    
    framelets = generate_framelets(im_ar, start_time, start_correction, frame_delay) # im_ar.astype(np.int) ** 2

    cam_pos, cam_orient = get_junocam_jupiter_rel_pos_orient(start_time, start_correction + 17 * frame_delay)

    y, x = np.mgrid[-y_dim/2:y_dim/2, -x_dim/2:x_dim/2]
    x += 260
    y += 95
    rays = np.concatenate([x[..., None], y[..., None], np.ones(((y_dim), (x_dim), 1)) * fl[0]], axis = -1)
    rays = rays.dot(cam_orient)

    surface_raster, surface_mask = project_onto_jupiter_surf(cam_pos, rays)
    
    np.save(meta_num + '-Raster', surface_raster)
    
    colors = np.zeros(((y_dim), (x_dim), 3))
    color_counts = np.zeros(((y_dim), (x_dim), 3))

    for k,framelet in enumerate(framelets):
        print(str(meta_num) + ': processing framelet', (k + 1), 'of', len(framelets))
        col = framelet.color
        brightnesses, valid_map = framelet.get_pixel_val_at_surf_point(surface_raster)
        colors[..., 2 - col] += brightnesses
        color_counts[..., 2 - col] += valid_map

    colors /= np.maximum(color_counts, 1)
    colors *= 255 / np.max(colors)

    colors = colors.astype(np.uint8)

    new_img = Image.fromarray(colors)
    new_img.save(str(meta_num) + '-Stitched.png')

In [9]:
## multithreaded execution - process all images in directory that have don't already have a stitched version
img_nums = []
for file in os.listdir('.'):
    if file.endswith('.json') and not os.path.isfile(file.split('-')[0] + '-Stitched.png'):
        img_nums.append(file.split('-')[0])
print_str = 'Processing the following images: '
for im in img_nums:
    print_str += (im + ', ')
print_str = print_str[:-2]
print(print_str)

num_cpus = multiprocessing.cpu_count() - 1
curr = 0

print("Begin stitching threads")

while curr < len(img_nums):
    threads = []
    num_threads = num_cpus if num_cpus < len(img_nums) else len(img_nums)
    print('Running ' + str(num_threads) + ' threads')
    for t in range(num_threads):
        try:
            threads.append(threading.Thread(target = stitch, args = (str(img_nums[curr]), )))
        except IndexError:
            print('\nStarting new round of threads')
        threads[t].start()
        curr += 1
    for t in threads:
        t.join()

print("End stitching threads")

Processing the following images: 6881, 6882, 6884, 6922, 6923
Begin stitching threads
Running 5 threads


Exception in thread Thread-9:
Traceback (most recent call last):
  File "/home/uzumochi/anaconda3/envs/spice/lib/python3.8/threading.py", line 932, in _bootstrap_inner
    self.run()
  File "/home/uzumochi/anaconda3/envs/spice/lib/python3.8/threading.py", line 870, in run
    self._target(*self._args, **self._kwargs)
  File "/tmp/ipykernel_3524/3987097976.py", line 21, in stitch
  File "/home/uzumochi/eigenjuno/STITCHING/stitch.py", line 310, in generate_framelets
    sun_pos = get_sun_jupiter_rel_pos(start_time, add_seconds=stripe_delay)
  File "/home/uzumochi/eigenjuno/STITCHING/stitch.py", line 87, in get_sun_jupiter_rel_pos
    et = spice.str2et(time_str) + add_seconds
  File "/home/uzumochi/anaconda3/envs/spice/lib/python3.8/site-packages/spiceypy/spiceypy.py", line 124, in with_errcheck
    check_for_spice_error(f)
  File "/home/uzumochi/anaconda3/envs/spice/lib/python3.8/site-packages/spiceypy/spiceypy.py", line 107, in check_for_spice_error
    raise stypes.dynamically_instanti

6923: processing framelet 1 of 108
6881: processing framelet 1 of 120
6882: processing framelet 1 of 120
6884: processing framelet 1 of 120
6881: processing framelet 2 of 120
6882: processing framelet 2 of 120
6923: processing framelet 2 of 108
6884: processing framelet 2 of 120
6881: processing framelet 3 of 120
6882: processing framelet 3 of 120
6884: processing framelet 3 of 120
6923: processing framelet 3 of 108
6881: processing framelet 4 of 120
6882: processing framelet 4 of 120
6884: processing framelet 4 of 120
6923: processing framelet 4 of 108
6881: processing framelet 5 of 120
6882: processing framelet 5 of 120
6884: processing framelet 5 of 120
6881: processing framelet 6 of 120
6923: processing framelet 5 of 108
6882: processing framelet 6 of 120
6884: processing framelet 6 of 120
6881: processing framelet 7 of 120
6923: processing framelet 6 of 108
6882: processing framelet 7 of 120
6884: processing framelet 7 of 120
6881: processing framelet 8 of 120
6882: processing fra

In [10]:
## copy all processed images and their metadata + surface rasters to testing directory
for file in sorted(os.listdir('.')):
    if fnmatch.fnmatch(file, '*-Metadata.json') or fnmatch.fnmatch(file, '*-Raster.npy') \
        or fnmatch.fnmatch(file, '*-Stitched.png'):
        shutil.copy(file, '../../TEST/' + file)